In [99]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

In [4]:
from google.colab import files
uploaded = files.upload()

Saving reddit.csv to reddit (1).csv


### Reading Data and preprocessing

In [100]:
import pandas as pd

#Data Input and Preprocessing
data = pd.read_csv('reddit.csv')
#test = pd.read_csv('test2_19.csv')
for row in range(len(data["id"])):
    #print(data["id"][row])
    data["id"][row]=(data["id"][row].splitlines())
    for num in range(len(data["id"][row])):
        strin=''.join(data["id"][row][num].split())
        newstr='';
        for i in range(len(strin)-2):
            newstr+=strin[i+2]
        data["id"][row][num]=newstr

for row in range(len(data["hate_speech_idx"])):
        if isinstance(data["hate_speech_idx"][row],float):
            data["hate_speech_idx"][row]="[0]"
        
data_mod = [[]]

for row in range(len(data["text"])):
    #print(data["id"][row])
    data["text"][row]=(data["text"][row].splitlines())
    for num in range(len(data["text"][row])):
        strin=' '.join(data["text"][row][num].split())
        newstr='';
        for i in range(len(strin)-2):
            newstr+=strin[i+2]
        data["text"][row][num]=newstr
        text = False
        for i in range(len(data["hate_speech_idx"][row])):
            if (data["hate_speech_idx"][row][i]!='[' and data["hate_speech_idx"][row][i]!=' ') and (data["hate_speech_idx"][row][i]!=']' and data["hate_speech_idx"][row][i]!=','):
                if int(num)+1 == int(data["hate_speech_idx"][row][i]):
                    line=[]
                    line.append(newstr)
                    line.append(1)
                    data_mod.append(line)
                    text = True
                    break
        if text == False:
            line=[]
            line.append(newstr)
            line.append(0)
            data_mod.append(line)
            
        

data_mod.remove([])        
print(data_mod[0][0])

data = pd.DataFrame(data_mod, columns = ['Text', 'Label'])


 A subsection of retarded Hungarians? Ohh boy. brace for a livid Bulbasaur coming in here trying to hate a hole in some of her stupider countrymen.


In [101]:
data.head()

,Text,Label
0,A subsection of retarded Hungarians? Ohh boy....,1
1,Hiii. Just got off work. 444 is mainly the ty...,0
2,wow i guess soyboys are the same in every cou...,0
3,Owen Benjamin's soyboy song goes for every co...,0
4,"> ""y'all hear sumn?"" by all means I live in a...",0


In [102]:
data.Label.unique()

array([1, 0])

In [7]:
data.shape

(22324, 2)

In [103]:
pos = []
neg = []
for l in data.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [104]:
data['Pos']= pos
data['Neg']= neg

In [105]:
data.head()

,Text,Label,Pos,Neg
0,A subsection of retarded Hungarians? Ohh boy....,1,1,0
1,Hiii. Just got off work. 444 is mainly the ty...,0,0,1
2,wow i guess soyboys are the same in every cou...,0,0,1
3,Owen Benjamin's soyboy song goes for every co...,0,0,1
4,"> ""y'all hear sumn?"" by all means I live in a...",0,0,1


### Clean data

In [106]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))
print(data["Text_Clean"].head())

0     A subsection of retarded Hungarians Ohh boy b...
1     Hiii Just got off work 444 is mainly the typa...
2     wow i guess soyboys are the same in every cou...
3     Owen Benjamins soyboy song goes for every cou...
4      yall hear sumn by all means I live in a smal...
Name: Text_Clean, dtype: object


In [107]:
import nltk
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [18]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [19]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [20]:
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stoplist]

In [21]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [23]:
result = [' '.join(sen) for sen in filtered_words]

In [24]:
data['Text_Final'] = result

In [25]:
data['tokens'] = filtered_words

In [26]:
data = data[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]

In [27]:
data[:4]

,Text_Final,tokens,Label,Pos,Neg
0,subsection retarded hungarians ohh boy brace l...,"[subsection, retarded, hungarians, ohh, boy, b...",1,1,0
1,hiii got work 444 mainly typa guys imagine wri...,"[hiii, got, work, 444, mainly, typa, guys, ima...",0,0,1
2,wow guess soyboys every country,"[wow, guess, soyboys, every, country]",0,0,1
3,owen benjamins soyboy song goes every country ...,"[owen, benjamins, soyboy, song, goes, every, c...",0,0,1


### Split data into test and train

In [29]:
data_train, data_test = train_test_split(data, test_size=0.20, random_state=42)

In [30]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

424837 words total, with a vocabulary size of 33666
Max sentence length is 763


In [31]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

107121 words total, with a vocabulary size of 16294
Max sentence length is 623


### Load Google News Word2Vec model

In [35]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [37]:
print(word2vec["cat"].shape)

def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

(300,)


### Get Embeddings

In [38]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [39]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [40]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 33596 unique tokens.


In [41]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [42]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(33597, 300)


In [66]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(test_cnn_data.shape)


(4465, 50)


### Tokenize and Pad sequences



In [46]:
label_names = ['Pos', 'Neg']

In [47]:
y_train = data_train[label_names].values

In [80]:
x_train = train_cnn_data
y_tr = y_train
print(x_train.shape)

array = np.arange(len(y_tr))
print(array.shape)
array[0] = 1
cnt = 0
for x in (y_tr):
  if x[1] == 1:
    array[cnt] = 1
  else:
    array[cnt] = 0
  cnt = cnt + 1
print(array.shape)
y_tr = array


(17859, 50)
(17859,)
(17859,)
[1 1 1 0 1 1 1 1 1 1 1 1 1 1 0]


In [81]:
def recurrent_nn(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    model = Sequential()
    model.add(embedding_layer)
    model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # try using different optimizers and different optimizer configs
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    print('Summary of the built model...')
    print(model.summary())
    return model

# GRU

In [82]:
model = recurrent_nn(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Summary of the built model...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           10079100  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                32064     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 10,111,197
Trainable params: 32,097
Non-trainable params: 10,079,100
_________________________________________________________________
None


### Train GRU

In [83]:
num_epochs = 5
batch_size = 34
type(x_train)
print(x_train.shape)
print(y_tr.shape)

(17859, 50)
(17859,)


In [84]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/5
473/473 [==============================] - 38s 81ms/step - loss: 0.4256 - accuracy: 0.8318 - val_loss: 0.3488 - val_accuracy: 0.8819
Epoch 2/5
473/473 [==============================] - 38s 80ms/step - loss: 0.3478 - accuracy: 0.8748 - val_loss: 0.3422 - val_accuracy: 0.8785
Epoch 3/5
473/473 [==============================] - 38s 81ms/step - loss: 0.3365 - accuracy: 0.8777 - val_loss: 0.3345 - val_accuracy: 0.8813
Epoch 4/5
473/473 [==============================] - 38s 81ms/step - loss: 0.3301 - accuracy: 0.8804 - val_loss: 0.3296 - val_accuracy: 0.8819
Epoch 5/5
473/473 [==============================] - 38s 80ms/step - loss: 0.3235 - accuracy: 0.8827 - val_loss: 0.3296 - val_accuracy: 0.8841


### Test GRU

In [85]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

5/5 [==============================] - 1s 101ms/step


In [93]:
labels = [neg]
cnt = 0
for x in (predictions):
  if x >= 0.5:
    predictions[cnt] = 0
  else:
    predictions[cnt] = 1
  cnt = cnt + 1
print(predictions)

[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


In [94]:


from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
actual = data_test.Label
predicted = predictions
target_names = ['class 0(Non-Hate)', 'class 1(Hate)']
results = confusion_matrix(actual, predicted) 
  
print ('Confusion Matrix :')
print (results) 
print ('Accuracy Score :',accuracy_score(actual, predicted)) 
print ('Report : ')
print(classification_report(actual, predicted, target_names=target_names)) 
#### Computing the Confusion Matrix ####

Confusion Matrix :
[[3154  215]
 [ 320  776]]
Accuracy Score : 0.8801791713325868
Report : 
                   precision    recall  f1-score   support

class 0(Non-Hate)       0.91      0.94      0.92      3369
    class 1(Hate)       0.78      0.71      0.74      1096

         accuracy                           0.88      4465
        macro avg       0.85      0.82      0.83      4465
     weighted avg       0.88      0.88      0.88      4465



In [95]:
print("macro_avg_F1_score= "+str(f1_score(actual,predicted,average='macro')))
print("micro_avg_F1_score= "+str(f1_score(actual,predicted,average='micro')))
print("weighted_avg_F1_score= "+str(f1_score(actual,predicted,average='weighted')))

macro_avg_F1_score= 0.8327345450262689
micro_avg_F1_score= 0.8801791713325868
weighted_avg_F1_score= 0.8780842656294062
